In [14]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import utils
import json
import os
import git
import warnings
warnings.filterwarnings("ignore")

In [15]:
from dotenv import load_dotenv
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY

## Get the necessary data from Hopsworks

In [16]:
project = hopsworks.login()
fs = project.get_feature_store() 
# secrets = utils.secrets_api(project.name)

CITY = "dublin"
STATION = "HEUSTON BRIDGE (NORTH)"

# latitude =
# longitude =

today = datetime.datetime.now()

2025-01-08 17:05:58,501 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-08 17:05:58,564 INFO: Initializing external client
2025-01-08 17:05:58,566 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-08 17:05:59,936 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494


In [17]:
# Retrieve feature groups
bike_fg = fs.get_feature_group(
    name='bike_data',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)


## Clone and pull the repository with the bike data

In [18]:
# Configuration
REPO_URL = "https://github.com/MaxHalford/bike-sharing-history"
CLONE_DIR = "./bike_data/bike-sharing-history"
TARGET_CITY = "dublin"
FILE_NAME = "jcdecaux.geojson"

In [19]:
if not os.path.exists(CLONE_DIR):
    print("Cloning repository...")
    git.Repo.clone_from(REPO_URL, CLONE_DIR)
repo = git.Repo(CLONE_DIR)

# go to main and pull the latest changes
repo.git.checkout("main", force=True)
repo.remotes.origin.pull()

## Get the latest datetime present in the bike data

In [20]:
# last_bike_datetime = "2025-01-06 15:06:11 UTC"

bike_df = bike_fg.read()

last_bike_datetime = bike_df["datetime"].max()
last_bike_datetime = last_bike_datetime.strftime("%Y-%m-%d %H:%M:%S %Z")

last_bike_datetime

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.47s) 


'2025-01-08 15:00:00 UTC'

## Loop through the commits and convert the bike data into a dataframe

In [21]:
results = {}

# Populate the results dict with the stations
data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
if os.path.exists(data_file):
    with open(data_file, "r") as f:
        data = f.read()
        data = json.loads(data)
        for feature in data["features"]:
            name = feature["properties"]["name"]
            results[name] = []

In [22]:
start_time = datetime.datetime.now()
last_day_and_hour = None

for commit in repo.iter_commits():
    # Stop when we reach the earliest bike date
    if commit.committed_datetime <= datetime.datetime.strptime(last_bike_datetime, "%Y-%m-%d %H:%M:%S %Z").replace(tzinfo=datetime.timezone.utc):
        print("breaking at: ", commit.committed_datetime)
        break

    # Skip commits from today
    if commit.committed_datetime > today.replace(tzinfo=datetime.timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0):
        first_commited_datetime = repo.commit().committed_datetime
        continue

    day_and_hour = commit.committed_datetime.replace(minute=0, second=0, microsecond=0)
    # print("day_and_hour: ", day_and_hour, " - last_day_and_hour: ", last_day_and_hour)
    if day_and_hour == last_day_and_hour:
        continue
    last_day_and_hour = day_and_hour

    # Get the data for the commit
    print("Processing commit: ", commit.hexsha, " - ", commit.committed_datetime)
    try:
        repo.git.checkout(commit.hexsha, force=True)
    except Exception as e:
        print("Error checking out commit: ", e)
        break

    data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
    if os.path.exists(data_file):
        with open(data_file, "r") as f:
            data = f.read()
            data = json.loads(data)
            for feature in data["features"]:
                try:
                    results[feature["properties"]["name"]].append([feature["properties"]["available_bikes"], commit.committed_datetime])
                except KeyError:
                    continue

print(results)


breaking at:  2025-01-08 14:49:05+00:00
{'CLARENDON ROW': [], 'BLESSINGTON STREET': [], 'BOLTON STREET': [], 'GREEK STREET': [], 'CHARLEMONT PLACE': [], 'CHRISTCHURCH PLACE': [], 'HIGH STREET': [], 'CUSTOM HOUSE QUAY': [], 'EXCHEQUER STREET': [], 'DAME STREET': [], 'EARLSFORT TERRACE': [], 'ECCLES STREET': [], 'FITZWILLIAM SQUARE WEST': [], 'FOWNES STREET UPPER': [], 'HARDWICKE STREET': [], 'GEORGES QUAY': [], 'GOLDEN LANE': [], 'GRANTHAM STREET': [], 'HERBERT PLACE': [], 'JAMES STREET EAST': [], 'LEINSTER STREET SOUTH': [], 'TOWNSEND STREET': [], 'CUSTOM HOUSE': [], 'CATHAL BRUGHA STREET': [], 'MERRION SQUARE EAST': [], 'MERRION SQUARE WEST': [], 'MOLESWORTH STREET': [], 'MOUNTJOY SQUARE WEST': [], 'ORMOND QUAY UPPER': [], 'PARNELL SQUARE NORTH': [], 'PARNELL STREET': [], 'PEARSE STREET': [], "PRINCES STREET / O'CONNELL STREET": [], 'PORTOBELLO HARBOUR': [], 'SMITHFIELD': [], "ST. STEPHEN'S GREEN EAST": [], "ST. STEPHEN'S GREEN SOUTH": [], 'TALBOT STREET': [], 'WILTON TERRACE': [], 'J

In [25]:
# turn results into a dataframe
df_bike_today = pd.DataFrame()

for station, values in results.items():
    if len(values) > 0:
        df = pd.DataFrame(values, columns=["num_bikes_available", "datetime"])
        df["datetime"] = pd.to_datetime(df["datetime"], utc=True)
        df["station"] = station.replace(" ", "_")
        df_bike_today = pd.concat([df_bike_today, df])

# if empty, do nothing
if not df_bike_today.empty:
    df_bike_today.dropna(inplace=True)
    df_bike_today["num_bikes_available"] = df_bike_today["num_bikes_available"].astype("float32")
    df_bike_today = df_bike_today[df_bike_today['station'].isin([STATION.replace(" ", "_")])]
    df_bike_today


## Fetch the weather data for the same time period

In [26]:
forecast_df = utils.get_hourly_weather_forecast(CITY)
forecast_df = forecast_df.rename(columns={'date_x': 'datetime'})
forecast_df = forecast_df.drop(columns=['date_y', 'date_only'])
forecast_df.dropna(inplace=True)

print(forecast_df.empty)

forecast_df

features: {'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
params: {'latitude': 53.35, 'longitude': -6.26, 'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
Coordinates 53.5°N -6.25°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
False


,datetime,temperature_2m,apparent_temperature,rain,snowfall,wind_speed_10m,daylight_duration,rain_sum,city
0,2025-01-08 00:00:00+00:00,-1.25,-5.474496,0.0,0.0,12.324414,28021.865234,0.0,dublin
1,2025-01-08 01:00:00+00:00,-1.45,-5.587059,0.0,0.0,11.542478,28021.865234,0.0,dublin
2,2025-01-08 02:00:00+00:00,-1.55,-5.520480,0.0,0.0,10.440001,28021.865234,0.0,dublin
3,2025-01-08 03:00:00+00:00,-1.70,-5.586253,0.0,0.0,9.726665,28021.865234,0.0,dublin
4,2025-01-08 04:00:00+00:00,-2.00,-5.771173,0.0,0.0,8.669949,28021.865234,0.0,dublin
...,...,...,...,...,...,...,...,...,...
235,2025-01-17 19:00:00+00:00,5.05,1.534287,0.0,0.0,14.578890,29351.843750,0.0,dublin
236,2025-01-17 20:00:00+00:00,4.85,1.368568,0.0,0.0,14.277983,29351.843750,0.0,dublin
237,2025-01-17 21:00:00+00:00,4.60,1.185653,0.0,0.0,13.479583,29351.843750,0.0,dublin
238,2025-01-17 22:00:00+00:00,4.30,0.943740,0.0,0.0,12.682018,29351.843750,0.0,dublin


## Insert the bike and weather data into Hopsworks

In [27]:
if df_bike_today.empty:
    print("No bike data available for today")
else:
    bike_fg.insert(df_bike_today)

No bike data available for today


In [28]:
if forecast_df.empty:
    print("No weather forecast available for today")
else:
    weather_fg.insert(forecast_df, write_options={"wait_for_job": True})


Uploading Dataframe: 100.00% |██████████| Rows 240/240 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/weather_data_1_offline_fg_materialization/executions
2025-01-08 17:07:25,548 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-01-08 17:07:28,719 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-01-08 17:09:13,659 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-01-08 17:09:16,839 INFO: Waiting for execution to finish. Current state: FINISHED. Final status: SUCCEEDED
2025-01-08 17:09:17,228 INFO: Waiting for log aggregation to finish.
2025-01-08 17:09:17,229 INFO: Execution finished successfully.
